In [1033]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [1034]:
data = pd.read_csv('movie_bd_v5.xls')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
301,tt1130080,22000000,35424826,The Informant!,Matt Damon|Scott Bakula|Joel McHale|Melanie Ly...,Steven Soderbergh,Based on a tattle-tale.,A rising star at agri-industry giant Archer Da...,108,Comedy|Crime|Drama|Thriller,Section Eight|Groundswell Productions|Warner B...,9/18/2009,6.0,2009
1500,tt0422295,16800000,2281089,Fur: An Imaginary Portrait of Diane Arbus,Nicole Kidman|Robert Downey Jr.|Ty Burrell|Har...,Steven Shainberg,A love story.,In 1958 New York Diane Arbus is a housewife an...,122,Drama|Mystery|Romance,River Road Entertainment|Edward R. Pressman Fi...,8/30/2006,6.3,2006
1589,tt0346156,70000000,57958696,Sky Captain and the World of Tomorrow,Jude Law|Gwyneth Paltrow|Giovanni Ribisi|Angel...,Kerry Conran,Who will save us?,"When gigantic robots attack New York City, ""Sk...",107,Mystery|Action|Thriller|Science Fiction|Adventure,Paramount Pictures|Natural Nylon Entertainment...,9/17/2004,5.7,2004
104,tt3623726,18000000,41325328,Ricki and the Flash,Meryl Streep|Kevin Kline|Mamie Gummer|Sebastia...,Jonathan Demme,Get Ready for Ricki.,"Meryl Streep stars as Ricki Rendazzo, a guitar...",100,Comedy|Drama|Music,Columbia TriStar|Clinica Estetico|LStar Capital,7/10/2015,5.5,2015
675,tt1034303,32000000,51155219,Defiance,Daniel Craig|Liev Schreiber|Jamie Bell|Alexa D...,Edward Zwick,Freedom begins with an act of defiance!,"Based on a true story, during World War II, fo...",137,Action|Drama|History|War,Paramount Vantage|Grosvenor Park Productions|B...,12/31/2008,6.6,2008


In [1035]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [1036]:
answers = {} # создадим словарь для ответов

data['release_date'] = pd.to_datetime(data['release_date']) 
# перевод значений столбца с датами в формат "datatime"


def convert_to_seasons(datetime):
    '''Функция для конвертации значения datetime в значение времени года'''
    if datetime.month in [12,1,2]:
        return 'winter'
    
    if datetime.month in [3,4,5]:
        return 'spring'
    
    if datetime.month in [6,7,8]:
        return 'summer'
    
    if datetime.month in [9,10,11]:
        return 'fall'
    return 'unknown'


# Для разных задач потребуется создание дополнительных столбцов:
data['profit'] = data['revenue'] - data['budget'] 
# создадим столбец, отражающий прибыль(убытки) каждого фильма
data['director_col'] = data.director.str.split('|')
# создаем столбец с помощью преобразования 
# строки с режиссерами фильма в список режиссеров фильма
data['date_seasons'] = data['release_date'].apply(convert_to_seasons, convert_dtype=True)
# создаем столбец, отражающий время года выхода фильма на экран
data['cast_col'] = data['cast'].str.split('|')
# создаем столбец с помощью преобразования строки 
# с актерами из фильма в список актеров из фильма
data['production_col'] = data['production_companies'].str.split('|')
# создаем столбец с помощью преобразования строки 
# со студиями фильма в список студий фильма 
data['original_title_len'] = data['original_title'].apply(len, convert_dtype=True)
# создаем столбец, отражающий количество символов в названии фильма
data['overview_words'] = data['overview'].apply(lambda x: len(x.split(' ')))
# создаем столбец, отражающий количество слов в описании фильма

  
def transform_nesting(nested_lists, list_values=[]):
    '''Функция для конвертации 2-х столбцов DataFrame, 
    в одном из которых — список значений, 
    во втором — связанные с 1 столбцом значения в серию,
    где индексы — сгруппированные данные из всех списков 1 столбца,
    а значения — просуммированные в соответствии с группировкой значения 2 столбца.
    Eсли параметр 2го столбца в функцию не был передан, 
    его заменяем на столбец со значениями : 1 '''
    
    if len(list_values) == 0: 
        list_values = [1 for i in range(0, len(nested_lists))]
    count = 0  # чтобы не "потерять" соответствие между 1 и 2 столбцом
    new_dict = {}  # для нужного нам группирования значений используем словарь
    
    for n_list in nested_lists: # "раскрываем" каждый список из столбца
        for num in n_list: # "раскрываем" каждое значение из списка
            if num not in new_dict.keys():
                new_dict[num] = 0 # чтобы избежать обращения к несуществующему ключу
            new_dict[num]+= list_values[count]  # суммируем текущее значение с предыдущими
        count+=1 
    return pd.Series(new_dict) #  на выходе объект Series, где 
                               #каждое значение из списка связаны со значением из 2 столбца

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [1037]:
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# +

In [1038]:
display(data[data['budget'] == data['budget'].max()].original_title.iloc[0])
# фильтруем DataFrame по максимальному значению бюджета

'Pirates of the Caribbean: On Stranger Tides'

ВАРИАНТ 2

In [1039]:
display(data.query('budget == budget.max()').original_title.iloc[0])

'Pirates of the Caribbean: On Stranger Tides'

# 2. Какой из фильмов самый длительный (в минутах)?

In [1040]:
answers['2'] = '2. Gods and Generals (tt0279111)'
#+

In [1041]:
display(data[data['runtime'] == data['runtime'].max()].original_title.iloc[0])
# фильтруем DataFrame по максимальному значению длительности фильма

'Gods and Generals'

# 3. Какой из фильмов самый короткий (в минутах)?





In [1042]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'
#+

In [1043]:
display(data[data['runtime']==data['runtime'].min()].original_title.iloc[0])
# фильтруем DataFrame по минимальному значению длительности фильма

'Winnie the Pooh'

# 4. Какова средняя длительность фильмов?


In [1044]:
answers['4'] = '2. 110'
#+

In [1045]:
display(round(data['runtime'].mean()))
# вычиcляем среднее значение по длительностям фильмов и округляем как в ответах

110

# 5. Каково медианное значение длительности фильмов? 

In [1046]:
answers['5'] = '1. 107'
#+

In [1047]:
display(round(data['runtime'].median()))
# вычиляем медианное значение по длительностям фильмов и округляем как в ответах

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [1048]:
answers['6'] = '5. Avatar (tt0499549)'
#+

In [1049]:
display(data[data['profit'] == data['profit'].max()].original_title.iloc[0])

'Avatar'

# 7. Какой фильм самый убыточный? 

In [1050]:
answers['7'] = '5. The Lone Ranger (tt1210819)'
#+

In [1051]:
display(data[data['profit'] == data['profit'].min()].original_title.iloc[0])

'The Lone Ranger'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [1052]:
answers['8'] = '1. 1478'
#+

In [1053]:
display(data[data['profit'] > 0]['imdb_id'].nunique()) 
# в условии задачи "завуалировано" понятие прибыль
# обращаемся к столбцу с id, чтобы посчитать количество уникальных значений

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [1054]:
answers['9'] = '4. The Dark Knight (tt0468569)'
#+

In [1055]:
group_years = data.groupby(['release_year'])['revenue'].max()
# группировка по году выпуска фильма и максимальному значению по кассовым сборам 
display(data[data.revenue == group_years[2008]].original_title.iloc[0])
# фильтруем данные по 2008 году, выбираем значение "названия фильма"

'The Dark Knight'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [1056]:
# ответ можно дать сразу(см. задачу 7)
answers['10'] = '5. The Lone Ranger (tt1210819)'
#+

In [1057]:
group_years = data.query('2011<release_year<2015')\
.groupby(['release_year'])['profit'].min()
# фильтруем данные по "нужным" годам и группируем фильмы по году выпуска и минимальной прибыли
min_profit = group_years.min()
display(data[data.profit == min_profit].original_title.iloc[0])
# фильтруем данные по минимальной прибыли из минимальных

'The Lone Ranger'

# 11. Какого жанра фильмов больше всего?

In [1058]:
answers['11'] = 'Drama'
# "+"
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# судьба не готовила нас к этому жуткому знаку "|"!
# функция transform_nesting полностью описана в "Предобработке"

In [1059]:
data_genres = data.genres.str.split('|') 
# преобразование данных о жанрах в серию списков
s_result = transform_nesting(data_genres) 
# подсчет количества жанров, аналог функции value_counts() для "сложных" данных
# так как в этой задаче необходимо работать с одним столбцом, 
# передаем и в transform_nesting 1 столбец
display(s_result[s_result == s_result.max()].index[0])
# получаем название жанра, встречающееся чаще всего в исходных данных.

'Drama'

ВАРИАНТ 2

In [1060]:
answers['11'] = '3. Drama'

In [1061]:
s_result = pd.Series(Counter(data_profit.genres.str.split('|').sum()))
# здесь используем Counter для подсчета количества "появлений" жанров в столбце
display(s_result[s_result == s_result.max()].index[0])

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [1062]:
answers['12'] = '1. Drama'

In [1063]:
data_profit = data[data.profit > 0]
# сначала отсекаем неприбыльные фильмы
s_result = pd.Series(Counter(data_profit.genres.str.split('|').sum()))
# здесь используем Counter для подсчета количества "появлений" жанров в столбце
display(s_result[s_result == s_result.max()].index[0])

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [1064]:
answers['13'] = '5. Peter Jackson'

In [1065]:
directors = transform_nesting(data.director_col,data.revenue)
#  функция transform_nesting для "связки" каждого режиссера с кассовыми сборами фильма
display(directors[directors == directors.max()].index[0])

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [1066]:
answers['14'] = '3. Robert Rodriguez'


In [1067]:
data_action = data[data.genres.str.contains('Action')]
# фильтруем  по жанру кино: Action
s = pd.Series(Counter(data_action.director.str.split('|').sum()))
# здесь используем Counter для подсчета количества "появлений" режиссеров в столбце.
# в итоге получаем объект Series с количествами фильмов снятых каждым режиссером
display(s[s == s.max()].index[0])

'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [1068]:
answers['15'] = '3. Chris Hemsworth'

In [1069]:
data_2012=data[data['release_date'].dt.year == 2012]\
.sort_values(['revenue'],ascending=False).head(len(data)//10).reset_index()
# Фильтруем по 2012 году, сортируем по кассовым сборам по убыванию 
# и берем 10% лучших по кассовым сборам фильмов
display(Counter(data_2012.cast.str.split('|').sum()).most_common())
# здесь используем Counter для подсчета количества "появлений" актеров в столбце
# Выбираем актера, который в начале списка и есть в одном из ответов в викторине, 
# так как задача не 100% конкретная

[('Bruce Willis', 5),
 ('Chris Hemsworth', 4),
 ('Elizabeth Banks', 4),
 ('Bryan Cranston', 4),
 ('Channing Tatum', 4),
 ('Anna Kendrick', 4),
 ('Kristen Stewart', 3),
 ('Sacha Baron Cohen', 3),
 ('Josh Hutcherson', 3),
 ('Guy Pearce', 3),
 ('Taylor Kitsch', 3),
 ('Brendan Gleeson', 3),
 ('Kate Beckinsale', 3),
 ('Paul Rudd', 3),
 ('Ralph Fiennes', 2),
 ('Anne Hathaway', 2),
 ('Tom Hardy', 2),
 ('John Leguizamo', 2),
 ('Denis Leary', 2),
 ('Seann William Scott', 2),
 ('Robert Pattinson', 2),
 ('Rhys Ifans', 2),
 ('Frances McDormand', 2),
 ('Jennifer Lawrence', 2),
 ('Woody Harrelson', 2),
 ('Tommy Lee Jones', 2),
 ('Emma Thompson', 2),
 ('Mark Wahlberg', 2),
 ('Giovanni Ribisi', 2),
 ('Billy Connolly', 2),
 ('Jane Lynch', 2),
 ('Hugh Jackman', 2),
 ('Kerry Washington', 2),
 ('Michael Fassbender', 2),
 ('Charlize Theron', 2),
 ('Liam Neeson', 2),
 ('Maggie Grace', 2),
 ('Adam Sandler', 2),
 ('Jason Statham', 2),
 ('Chris Pine', 2),
 ('Alec Baldwin', 2),
 ('Brooklyn Decker', 2),
 ('Sam W

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [1070]:
answers['16'] = '3. Matt Damon'

In [1071]:
data_budget = data[data['budget'] > data['budget'].mean()]
# отсекаем фильмы с бюджетом "ниже среднего"
Counter(data_budget.cast.str.split('|').sum()).most_common()[0][0]
# считаем в скольких высокобюджетных фильмах снялся каждый актер 
# и выделяем одного очень прилично зарабатывающего

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [1072]:
answers['17'] = '3. Matt Damon'
#+

In [1073]:
data_Cage = data[data.cast.str.contains('Nicolas Cage')]
# ищем все фильмы с упоминаниями Кейджа в актерах
Counter(data_Cage.genres.str.split('|').sum()).most_common()[0][0]
# выбираем самый частый жанр для Nicci

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [1074]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'
#+

In [1075]:
data_paramount = data[data.production_companies.str.contains('Paramount Pictures')]
# ищем все фильмы с упоминаниями студии Paramount Pictures
data_paramount[data_paramount['profit'] == data_Paramount['profit'].min()]\
.original_title.any()
# выделяем минимальных по прибыльности фильм

'K-19: The Widowmaker'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [1076]:
answers['19'] = '5. 2015'
# "+"

In [1077]:
s = data.groupby(data['release_date'].dt.year)['revenue'].sum()
# Группируем данные по годам и максимальным кассовым сборам
display(s[s == s.max()].index[0])

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [1078]:
answers['20'] = '1. 2014'

In [1079]:
data_wb = data[data.production_companies.str.contains('Warner')]
# ищем все фильмы с упоминаниями студии Warner Bros
s=data_wb.groupby(data['release_date'].dt.year)['profit'].sum()
# группируем отфильтрованные  данные по годам и прибыли
display(s[s==s.max()].index[0])

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [1080]:
answers['21'] = '4. Сентябрь'

In [1081]:
s = data.groupby(data['release_date'].dt.month)['imdb_id'].count()
# группируем данные по месяцам и считаем количество фильмов
display(s[s == s.max()].index[0])

9

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [1082]:
answers['22'] = '2. 450'

In [1083]:
s = data.groupby(data['release_date'].dt.month)['imdb_id'].count()
# группируем данные по месяцам и считаем количество фильмов
display(s[5:8].sum())
# выделяем летние месяцы и суммируем количество фильмов

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [1084]:
answers['23'] = '5. Peter Jackson'

In [1085]:
data_seasons = data[['director','date_seasons']]
# выделяем для удобства 2 столбца, с режиссерами и подготовленный в предобработке столбец с временами года
data_seasons_analysis = pd.get_dummies(data_seasons, columns=['date_seasons'])
# вызываем функцию get_dummies() для создания столбцов по каждому времени года
find_winter = data_seasons_analysis.\
groupby(['director'])['date_seasons_winter'].sum().reset_index()
# группируем получившуюся таблицу по режиссерам и зиме
find_winter['director'] = find_winter['director']\
.apply(lambda x : x.split('|'), convert_dtype=True)
# преобразуем столбец со "строковыми" режиссерами в список режиссеров
s_result = transform_nesting(find_winter.director, find_winter.date_seasons_winter)
# для каждого режиссера считаем, сколько фильмов он выпустил зимой
s_result[s_result==s_result.max()].index[0]

'Peter Jackson'

ВАРИАНТ 2

In [1086]:
# "Простое решение", но не 100% решение, так как "объединенные" режиссеры, 
# но ответ сразу ясен + есть инф-я относительно остальных времен года, 
# что может подразумеваться в условии задачи
new_data = data.groupby(['director'])['date_seasons'].value_counts()
display(new_data.sort_values(ascending=False))

director                       date_seasons
Peter Jackson                  winter          7
Clint Eastwood                 winter          6
Steven Soderbergh              winter          6
Marc Forster                   fall            6
Michael Bay                    summer          6
                                              ..
Michael Lembeck                winter          1
Michael McCullers              spring          1
Michael Moore                  fall            1
                               spring          1
Aaron Seltzer|Jason Friedberg  winter          1
Name: date_seasons, Length: 1446, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [1087]:
answers['24'] = '5. Four By Two Productions'

In [1088]:
film_len = transform_nesting(data.production_col,data.original_title_len)\
.reset_index()
# получаем Series общих длин фильмов, снятых на каждой студии, 
# столбец с количеством символов в названии фильма задан в предобработке
film_count = transform_nesting(data.production_col).reset_index()
# получаем Series с общим количеством фильмов, снятых на каждой студии 
merge_production = pd.merge(film_len,film_count, on='index', how='outer')
# соединяем 2 Series по именам студий
merge_production.columns=['production','len','count']
# переименовываем столбцы для удобства
merge_production['mean']=merge_production['len']/merge_production['count']
# вычисляем среднюю по студии длину названия фильма по количеству символов
display(merge_production[merge_production['mean'] == merge_production['mean'].max()]\
        ['production'].any())
# выбираем студию, у которой в среднем самые длинные названия фильмов

'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [1089]:
answers['25'] = '3. Midnight Picture Show'
#"+"

In [1090]:
film_overview_words = transform_nesting(data.production_col, data.overview_words)\
.reset_index()

film_count = transform_nesting(data.production_col).reset_index()
merge_overview = pd.merge(film_overview_words, film_count, on='index', how='outer')
merge_overview.columns = ['production','overview_words','count']

merge_overview['mean'] = merge_overview['overview_words'] / merge_production['count']
# аналогично заданию 24, столбец количества слов в описании фильма задан в предобработке
display(merge_overview[merge_overview['mean'] == merge_overview['mean'].max()]\
        ['production'].any())

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [1091]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'
#"+"

In [1092]:
list(data.sort_values(by=['vote_average'], ascending=False).reset_index()\
     .head(len(data)//100)['original_title'])
# сортируем по рейтингу и выбираем 1/100 от исходных данных

['The Dark Knight',
 'Interstellar',
 'The Imitation Game',
 'Inside Out',
 'Room',
 'The Wolf of Wall Street',
 'Gone Girl',
 '12 Years a Slave',
 'Guardians of the Galaxy',
 'The Lord of the Rings: The Return of the King',
 'Memento',
 'Inception',
 'The Pianist',
 'The Grand Budapest Hotel',
 'Her',
 'Spotlight',
 'Big Hero 6',
 'The Fault in Our Stars']

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [1093]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'
#"+"

In [1097]:
list_actors = list(Counter(data['cast'].str.split('|').sum()).keys())
# получаем с помощью Counter список всех актеров
result=pd.Series([0,0])

for actor in list_actors:
    data_actor = data[data.cast.str.contains(actor)] 
    # ограничиваем поиск связей только по фильмам, где снимался "текущий" актер
    counter_actor = Counter((data_actor.cast.str.split('|').sum()))
    counter_actor[actor] = 0  
    # "обнуляем" значение пересечения актера с самим собой, оно будет минимальным в словаре
    counter_s = pd.Series(counter_actor).sort_values()
    if counter_s[-1] > result[1]:
        result = counter_s[::len(counter_s)-1]
    # выбираем max значение пересечения 2 актеров во всех фильмах среди всех актеров по ходу цикла 
    # выбираем 1 значение - min и последнее - max
    
# Создаем словарь в цикле: 
# каждый актер - ключ в словаре, 
# значения - вложенные словари, 
# их ключи - другие актеры, пересекающиеся в фильмах,
# значение вложенных словарей - количество пересечений в фильмах

print(result.index[0],'&',result.index[1])


Daniel Radcliffe & Rupert Grint


ВАРИАНТ 2

In [1098]:
# О нет! Хоть режьте! =О

# Submission

In [1099]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '3. Matt Damon',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [1100]:
# и убедиться что ни чего не пропустил)
len(answers)

27